# Roofline model

The [roofline model](https://en.wikipedia.org/wiki/Roofline_model) is a relatively new idea on how to evaluate the performance of an alogithm. Assuming a simplistic view of a CPU, all algorithms are constrained either by bandwidth-to-memory or floating-point operations per second (FLOPS). We can plot both of these limits on the same plot by multiplying bandwidth by operational intensity. Operational intensity is the number of bytes that are read from memory per floating-point operation. In the next cell, you can plot the roofline model for the node you are running on for this lab.

In [ ]:
%matplotlib inline

import numpy
import math
import matplotlib.pyplot as plt

ops = numpy.empty([7])
bandwidth = numpy.empty([7])
flops = numpy.empty([7])

for i in range(7):
    ops[i] = math.pow(2, i - 3)
    bandwidth[i] = ops[i] * 11.000
    flops[i] = 2.300
    
plt.loglog(ops, bandwidth, basex=2, basey=2)
plt.loglog(ops, flops, basex=2, basey=2)
plt.xlabel("Operational intensity")
plt.ylabel("GFLOPS")
plt.legend(("Bandwidth * Operational Intensity", "Floating point operations"))

Notice that the cross-over point is around $\frac{1}{4}$.  This is actually far from accurate because our simplified view does not account for the parallelism available on the machine along with the different memory levels of the chip.

## Part 1: Calculating operational intensity

In this lab we will be solving the heat equation using an explicit scheme.  Our implicit equation for updating the heat $H$ by  $\Delta t$ at a given $x$ and $y$ with some conductivity $c$ is:


\begin{equation}
\frac{H(x,y,t+1)-H(x,y,t)}{\Delta t} = c(x,y)\left(\frac{2*H(x,y,t)-H(x-1,y,t)-H(x+1,y,t)}{\Delta x^2} 
+ \frac{2*H(x,y,t)-H(x,y+1,t)-H(x,y-1,t)}{\Delta y^2} \right).
\end{equation}


With equal spacing in $x$ and $y$ we can set:


\begin{equation}
d(x,y)=\frac{c(x,y)*\Delta t}{\Delta x^2},
\end{equation}


which simplifies our updating scheme to:


\begin{equation}
H(x,y,t+1)= d(x,y) \left(4*H(x,y,t)-H(x-1,y,t)-H(x+1,y,t) -H(x,y+1,t)-H(x,y-1,t) \right) +  H(x,y,t) .
\end{equation}


One of the dumbest ways we can solve this problem is to make a matrix $m$ of dimensions $(nx,ny,nx,ny)$, and then fill it with zeros at every location except at the five points on the right-hand side of the above equation. The advantage of this approach is that it leads to a pretty simple updating algorithm to caculate algorithmic intensity.  Below is a Pythonized version of this update rule.

In [ ]:
def update(sout, sin, m):
    for iyo in range(sout.shape[0]):
        for ixo in range(sout.shape[1]):
            sout[iyo,ixo] = sin[iyo,ixo] #previous value
            for iyi in range(sout.shape[0]):
                for ixi in range(sout.shape[1]):
                    sout[iyo,ixo] += sin[iyi,ixi] * m[iyo,ixo,iyi,ixi]

Assuming $nx$ and $ny$ are large enough, all we need to do is assess the inner-most line in the loop.  In this case we are doing two multiplications and an addition, so a total of three floating-point operations.  At each location we are reading two floats (so eight bytes).  So our operational intensity is $I=\frac{3}{8}$. In terms of operational intensity this approach isn't terrible. The problem is the total number of operations we are doing. Each time step involves $n1*n1*n2*n2*3$ operations, the vast majority of which are just zeros. Below is a C++ implementation.

In [ ]:
%%writefile src/lib/heat.cc
#include "heat.h"
#include <stdlib.h>
#include <time.h>
#include <iostream>

void heatEq::allocFill(const std::shared_ptr<float2DReg> in) {
  _n1 = in->getHyper()->getAxis(1).n;
  _n2 = in->getHyper()->getAxis(2).n;
  _dif.resize(_n1 * _n2);
  srand(time(NULL));
    
  for (int i = 0; i < _n1 * _n2; i++) {
    _dif[i] = .05 + .1 * (((float)rand() / (float)RAND_MAX) - .5);
  }
}

heatEqFull::heatEqFull(const std::shared_ptr<float2DReg> in) {
  allocFill(in);
  for (int i2 = 0; i2 < _n2; i2++) {
    for (int i1 = 0; i1 < _n1; i1++) {
      std::vector<float> v(_n1 * _n2, 0.);
      if (i2 > 1 && i2 < _n2 - 1 && i1 > 1 && i1 < _n1 - 1) {
        v[i1 + i2 * _n1] = -4 * _dif[i1 + i2 * _n1] + 1;
        v[i1 + i2 * _n1 - 1] = _dif[i1 + i2 * _n1];
        v[i1 + i2 * _n1 + 1] = _dif[i1 + i2 * _n1];
        v[i1 + i2 * _n1 + _n1] = _dif[i1 + i2 * _n1];
        v[i1 + i2 * _n1 - _n1] = _dif[i1 + i2 * _n1];
      }
      _full.push_back(v);
    }
  }
}

void heatEqFull::explicitStep(const std::shared_ptr<float2DReg> in,
                              std::shared_ptr<float2DReg> out) {
  int i = 0;
  for (int i2 = 0; i2 < _n2; i2++) {
    for (int i1 = 0; i1 < _n1; i1++, i++) {
      (*out->_mat)[i2][i1] = 0;
      int j = 0;
      for (int j2 = 0; j2 < _n2; j2++) {
        for (int j1 = 0; j1 < _n1; j1++, j++) {
          (*out->_mat)[i2][i1] += (*in->_mat)[j2][j1] * _full[i][j];
        }
      }
    }
  }
}

heatEqSparse::heatEqSparse(const std::shared_ptr<float2DReg> in) {
  allocFill(in);
}
        
void heatEqSparse::explicitStep(const std::shared_ptr<float2DReg> in,
                                std::shared_ptr<float2DReg> out) {}

heatEqOp::heatEqOp(const std::shared_ptr<float2DReg> in) { 
  allocFill(in);
}
        
void heatEqOp::explicitStep(const std::shared_ptr<float2DReg> in,
                            std::shared_ptr<float2DReg> out) {}

Compile the code using the cell below.

In [ ]:
!mkdir -p build; cd build; rm -rf *; cmake -DgenericIO_DIR=/opt/SEP/lib/cmake -DCMAKE_INSTALL_PREFIX=/opt/SEP/local -DCMAKE_CXX_COMPILER="g++" -DCMAKE_CXX_FLAGS="-O2 -fno-tree-vectorize" ../src; make install

Now we initalize the operator.

In [ ]:
# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

import Hypercube
import SepVector
import LabRoofline

dom1 = SepVector.getSepVector(Hypercube.hypercube(ns=[100,100]))

op = LabRoofline.heatEqFull(dom1)

And now we can run 200 steps of the heat equation. (**Warning:** Be patient, this may take a little while).

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

dom2 = dom1.clone()
dom1.zero()
dom1.getNdArray()[50][50] = 1.

for i2 in range(100):
    op.explicitStep(dom1, dom2)
    op.explicitStep(dom2, dom1)

op.explicitStep(dom1, dom2)
plt.imshow(dom2.getNdArray())

Try out different sizes of the domain. Is the problem memory-bound or compute-bound? Does the result make sense given the operational intensity? How much of the peak performance are you achieving?

# Part 2

We could also do the explicit update step by sparse matrix multiplication. We could create integer arrays where our matrix is non-zero and then have our explicit step be a sparse-matrix multiplication. Code this approach in the class `heatEqSparse`. Plot the result to make sure your answer makes sense.

Calculate the operational intensity of the sparse matrix multiplication. 

What percentage of maximum performance did you achieve?

# Part 3

Another approach is to directly code the explicit step in operator form. Instead of coding the operator in a full, or sparse matrix, directly apply the derivatives.  Code this approach in the class `heatEqOp`. 

Calculate the operational intensity of this approach.

How does it compare to ideal performance?

# Part 4

Suggest ways to improve your approach in Part 3 and 4 to achieve better performance.